In [1]:
import torch
from torch.nn.functional import conv2d

import time
import cv2
import numpy as np
from scipy import signal
from scipy import ndimage

In [19]:
# a = torch.tensor([[2, 5, 6, 9],
#                  [10, 1, 3, 4],
#                  [8, 7, 0, -1],
#                  [-7, -4, -2, 8]]).unsqueeze(0).unsqueeze(0)

# a = torch.tensor([[2, 5, 6],
#                  [10, 1, 3],
#                  [8, 7, 0]]).unsqueeze(0).unsqueeze(0)

# a = torch.tensor(np.arange(1, 50).reshape((7, 7))).unsqueeze(0).unsqueeze(0)
a = torch.tensor(np.arange(1, 82).reshape((9, 9))).unsqueeze(0).unsqueeze(0)

# k1_11 = torch.tensor([[0, 1, 0],
#                      [1, 2, 1],
#                      [0, 1, 0]]).unsqueeze(0).unsqueeze(0)
k1_11 = torch.tensor([[0, 1, 0],
                     [1, 2, 1],
                     [0, 1, 3]]).unsqueeze(0).unsqueeze(0)
k1_21 = torch.tensor([[-1, 0, -1],
                     [3, 1, 3],
                     [-1, 0, -1]]).unsqueeze(0).unsqueeze(0)

# k2_11 = torch.tensor([[3, 2, 3],
#                      [0, 1, 0],
#                      [3, 2, 3]]).unsqueeze(0).unsqueeze(0)
k2_11 = torch.tensor([[5, 2, 7],
                     [0, 1, 0],
                     [3, 2, 9]]).unsqueeze(0).unsqueeze(0)
# k2_12 = torch.tensor([[1, 2, 1],
#                      [1, -2, 1],
#                      [1, 2, 1]]).unsqueeze(0).unsqueeze(0)
k2_12 = torch.tensor([[1, 2, 1],
                     [1, -2, 1],
                     [1, 2, 4]]).unsqueeze(0).unsqueeze(0)
# k2_21 = torch.tensor([[1, 1, 1],
#                      [2, 3, 2],
#                      [1, 1, 1]]).unsqueeze(0).unsqueeze(0)
k2_21 = torch.tensor([[3, 1, 12],
                     [2, 3, 7],
                     [9, 1, 10]]).unsqueeze(0).unsqueeze(0)
# k2_22 = torch.tensor([[-2, 3, -2],
#                      [1, 1, 1],
#                      [-2, 3, -2]]).unsqueeze(0).unsqueeze(0)
k2_22 = torch.tensor([[-4, 7, -10],
                     [1, 1, 1],
                     [-20, 3, -32]]).unsqueeze(0).unsqueeze(0)

k3_11 = torch.tensor([[2, 1, 2],
                     [-1, 0, -1],
                     [2, 1, 2]]).unsqueeze(0).unsqueeze(0)
k3_12 = torch.tensor([[4, 1, 4],
                     [-4, -1, -4],
                     [4, 1, 4]]).unsqueeze(0).unsqueeze(0)
k3_21 = torch.tensor([[3, 3, 3],
                     [-1, -1, -1],
                     [3, 3, 3]]).unsqueeze(0).unsqueeze(0)
k3_22 = torch.tensor([[2, -3, 2],
                     [1, 5, 1],
                     [2, -3, 2]]).unsqueeze(0).unsqueeze(0)


# tensors = [a, k1_11, k1_21, k2_11, k2_12, k2_21, k2_22, k3_11, k3_12, k3_21, k3_22]
# for i in range(len(tensors)):
#     tensors[i] = tensors[i].unsqueeze(-1)

In [20]:
def C_torch(input, weight, padding='same'):
    return conv2d(input, weight, padding=padding)

def C_scipy(input, weight):
    if len(input.shape) == 4:
        input = input.squeeze()
    if len(weight.shape) == 4:
        weight = weight.squeeze()
    return signal.convolve2d(input, weight, mode='full', boundary='fill', fillvalue=0)

def C_scipy_same(input, weight):
    if len(input.shape) == 4:
        input = input.squeeze()
    if len(weight.shape) == 4:
        weight = weight.squeeze()
    return signal.convolve2d(input, weight, mode='same', boundary='fill', fillvalue=0)

C = C_scipy

In [21]:
# C(k1_11, k1_21)
# print(k1_11)
# print(k1_21)
# print(signal.convolve2d(k1_11.squeeze(), k1_21.squeeze(), mode='full', boundary='fill', fillvalue=0))
# print(signal.convolve2d(k1_11.squeeze(), k1_21.squeeze(), mode='same', boundary='fill', fillvalue=0))

In [22]:
# k1_11.shape
# k1_21.shape
print(k1_11.shape, k1_21.shape)

torch.Size([1, 1, 3, 3]) torch.Size([1, 1, 3, 3])


In [23]:
# b1 = C(k1_11, a)
# b2 = C(k1_21, a)
# c1 = C(k2_11, b1) + C(k2_12, b2)
# c2 = C(k2_21, b1) + C(k2_22, b2)
# d1 = C(k3_11, c1) + C(k3_12, c2)
# print(d1)

# d1_eq = C(C(k3_11, C(k2_11, k1_11) + C(k2_12, k1_21)) + C(k3_12, C(k2_21, k1_11) + C(k2_22, k1_21)), a)
# print(d1_eq)
C = C_torch
b1 = C(a, k1_11)
b2 = C(a, k1_21)
c1 = C(b1, k2_11) + C(b2, k2_12)
c2 = C(b1, k2_21) + C(b2, k2_22)
d1 = C(c1, k3_11) + C(c2, k3_12)
print(d1)

eq_k = C(k3_11, C(k2_11, k1_11) + C(k2_12, k1_21)) + C(k3_12, C(k2_21, k1_11) + C(k2_22, k1_21))
d1_eq = C(a, eq_k)
print(d1_eq)

print(eq_k)

C = C_scipy
b1 = C(a, k1_11)
b2 = C(a, k1_21)
c1 = C(b1, k2_11) + C(b2, k2_12)
c2 = C(b1, k2_21) + C(b2, k2_22)
d1 = C(c1, k3_11) + C(c2, k3_12)
print(d1)

eq_k = C(k3_11, C(k2_11, k1_11) + C(k2_12, k1_21)) + C(k3_12, C(k2_21, k1_11) + C(k2_22, k1_21))
d1_eq = C(a, eq_k)
print(d1_eq)

print(eq_k)

C = C_scipy_same
b1 = C(a, k1_11)
b2 = C(a, k1_21)
c1 = C(b1, k2_11) + C(b2, k2_12)
c2 = C(b1, k2_21) + C(b2, k2_22)
d1 = C(c1, k3_11) + C(c2, k3_12)
print(d1)

eq_k = C(k3_11, C(k2_11, k1_11) + C(k2_12, k1_21)) + C(k3_12, C(k2_21, k1_11) + C(k2_22, k1_21))
d1_eq = C(a, eq_k)
print(d1_eq)

print(eq_k)

tensor([[[[ 20358,  34092,  39851,  41881,  43902,  45923,  41130,  32829,
            16688],
          [ 39005,  63756,  75436,  78113,  81348,  84583,  76322,  61700,
            31659],
          [ 60876, 100445, 117587, 121754, 126263, 130772, 117394,  94500,
            48483],
          [ 83247, 136893, 159420, 163734, 168543, 173352, 155646, 124578,
            64488],
          [106548, 175053, 202728, 207015, 211824, 216633, 194337, 154872,
            80580],
          [129849, 213213, 246036, 250296, 255105, 259914, 233028, 185166,
            96672],
          [ 81337, 126498, 142362, 138465, 141462, 144459, 137925, 104689,
            60687],
          [170837, 293348, 337176, 351439, 356504, 361569, 319954, 255484,
           132165],
          [ -4466, -19784, -21633, -30649, -30602, -30555, -21129, -17308,
            -6059]]]])
tensor([[[[  4019,   7997,   7475,   6953,   6431,   5909,   5387,   4865,
             3452],
          [  4857,   8311,  10307,  12303,  142

In [24]:
# b1 = C(k1_11, a)
# b2 = C(k1_21, a)
# c1 = C(k2_11, b1) + C(k2_12, b2)
# c2 = C(k2_21, b1) + C(k2_22, b2)
# d1 = C(k3_11, c1) + C(k3_12, c2)
# print(d1)

# d1_eq = C(C(k3_11, C(k2_11, k1_11) + C(k2_12, k1_21)) + C(k3_12, C(k2_21, k1_11) + C(k2_22, k1_21)), a)
# print(d1_eq)
C = C_torch
b1 = C(a, k1_11)
b2 = C(a, k1_21)
c1 = C(b1, k2_11) + C(b2, k2_12)
print(c1)

eq_k = C(k2_11, k1_11) + C(k2_12, k1_21) #+ C(k2_21, k1_11) + C(k2_22, k1_21)
c1_eq = C(a, eq_k)
print(c1_eq)

print(eq_k)

C = C_scipy
b1 = C(a, k1_11)
b2 = C(a, k1_21)
c1 = C(b1, k2_11) + C(b2, k2_12)
print(c1)

eq_k = C(k2_11, k1_11) + C(k2_12, k1_21) #+ C(k2_21, k1_11) + C(k2_22, k1_21)
c1_eq = C(a, eq_k)
print(c1_eq)

print(eq_k)

C = C_scipy_same
b1 = C(a, k1_11)
b2 = C(a, k1_21)
c1 = C(b1, k2_11) + C(b2, k2_12)
print(c1)

eq_k = C(k2_11, k1_11) + C(k2_12, k1_21) #+ C(k2_21, k1_11) + C(k2_22, k1_21)
c1_eq = C(a, eq_k)
print(c1_eq)

print(eq_k)

tensor([[[[ 1604,  2128,  2310,  2465,  2620,  2775,  2930,  2071,   882],
          [ 3139,  4234,  4594,  4882,  5170,  5458,  5746,  4172,  1971],
          [ 4956,  6732,  7206,  7500,  7794,  8088,  8382,  6185,  2960],
          [ 6810,  9279,  9852, 10146, 10440, 10734, 11028,  8201,  3950],
          [ 8664, 11826, 12498, 12792, 13086, 13380, 13674, 10217,  4940],
          [10518, 14373, 15144, 15438, 15732, 16026, 16320, 12233,  5930],
          [12372, 16920, 17790, 18084, 18378, 18672, 18966, 14249,  6920],
          [11379, 15840, 16848, 17100, 17352, 17604, 17856, 14560,  7009],
          [ 6056,  9238,  9912, 10055, 10198, 10341, 10484,  8148,  4794]]]])
tensor([[[[  718,   996,  1121,  1246,  1371,  1496,  1621,  1746,  1183],
          [ 1709,  2266,  2463,  2660,  2857,  3054,  3251,  3448,  2347],
          [ 3059,  4039,  4236,  4433,  4630,  4827,  5024,  5221,  3535],
          [ 4409,  5812,  6009,  6206,  6403,  6600,  6797,  6994,  4723],
          [ 5759,  758

In [18]:
ver1 = torch.tensor([[-1, -2, -1],
                     [0, 0, 0],
                     [1, 2, 1]]).unsqueeze(0).unsqueeze(0)
ver2 = torch.tensor([[1, 2, 1],
                     [0, 0, 0],
                     [-1, -2, -1]]).unsqueeze(0).unsqueeze(0)
hor1 = torch.tensor([[-1, 0, 1],
                     [-2, 0, 2],
                     [-1, 0, 1]]).unsqueeze(0).unsqueeze(0)
hor2 = torch.tensor([[1, 0, -1],
                     [2, 0, -2],
                     [1, 0, -1]]).unsqueeze(0).unsqueeze(0)

# print(C_scipy(ver, hor1))
# print(C_scipy(ver, hor2))
print(C_scipy(C_scipy(ver1, hor1), C_scipy(ver2, hor2)), '\n')
print(C_scipy(C_scipy(ver1, hor2), C_scipy(ver2, hor1)), '\n')

[[  1   4   4  -4 -10  -4   4   4   1]
 [  4  16  16 -16 -40 -16  16  16   4]
 [  4  16  16 -16 -40 -16  16  16   4]
 [ -4 -16 -16  16  40  16 -16 -16  -4]
 [-10 -40 -40  40 100  40 -40 -40 -10]
 [ -4 -16 -16  16  40  16 -16 -16  -4]
 [  4  16  16 -16 -40 -16  16  16   4]
 [  4  16  16 -16 -40 -16  16  16   4]
 [  1   4   4  -4 -10  -4   4   4   1]] 

[[  1   4   4  -4 -10  -4   4   4   1]
 [  4  16  16 -16 -40 -16  16  16   4]
 [  4  16  16 -16 -40 -16  16  16   4]
 [ -4 -16 -16  16  40  16 -16 -16  -4]
 [-10 -40 -40  40 100  40 -40 -40 -10]
 [ -4 -16 -16  16  40  16 -16 -16  -4]
 [  4  16  16 -16 -40 -16  16  16   4]
 [  4  16  16 -16 -40 -16  16  16   4]
 [  1   4   4  -4 -10  -4   4   4   1]] 



In [79]:
c1 = C(k2_11, C(k1_11, a)) + C(k2_12, C(k1_21, a))
print(c1)   

c1 = C(C(k2_11, k1_11), a) + C(C(k2_12, k1_21), a)
print(c1)   
c1_eq = C(C(k2_11, k1_11) + C(k2_12, k1_21), a)
print(c1_eq)

kt = torch.tensor([[2, -3, 4],
                    [1, 5, 1],
                    [4, -3, 4]]).unsqueeze(0).unsqueeze(0)

# distributive - yes
c1 = C(a, C(k2_11, k1_11)) + C(a, C(k2_12, k1_21))
print(c1)   
c1_eq = C(a, C(k2_11, k1_11) + C(k2_12, k1_21))
print(c1_eq)

# distributive - yes
c1 = C(a, C(kt, kt)) + C(a, C(kt, kt))
print(c1)   
c1_eq = C(a, C(kt, kt) + C(kt, kt))
print(c1_eq)

tensor([[[[172, 252, 115],
          [405, 456, 349],
          [164, 264, 107]]]])
tensor([[[[ 54, 264, 221],
          [322, 456, 432],
          [ 94, 252, 189]]]])
tensor([[[[ 54, 264, 221],
          [322, 456, 432],
          [ 94, 252, 189]]]])
tensor([[[[189, 252,  94],
          [432, 456, 322],
          [221, 264,  54]]]])
tensor([[[[189, 252,  94],
          [432, 456, 322],
          [221, 264,  54]]]])
tensor([[[[-204, 1250,  816],
          [2232,   26, 1042],
          [ 852, 1498, -496]]]])
tensor([[[[-204, 1250,  816],
          [2232,   26, 1042],
          [ 852, 1498, -496]]]])


In [77]:
# associative - no
print(C(C(a, k1_11), k2_11))
print(C(a, C(k1_11, k2_11)))

# commutative - yes reversed
print(C(a, k1_11))
print(C(k1_11, a))

kt = torch.tensor([[2, -3, 4],
                    [1, 5, 1],
                    [4, -3, 4]]).unsqueeze(0).unsqueeze(0)

# commutative - yes reversed
print(C(a, kt))
print(C(kt, a))

tensor([[[[162, 205, 127],
          [261, 357, 199],
          [176, 209, 117]]]])
tensor([[[[165, 201, 112],
          [273, 357, 187],
          [215, 213,  90]]]])
tensor([[[[19, 19, 20],
          [31, 27, 13],
          [33, 23, 10]]]])
tensor([[[[10, 23, 33],
          [13, 27, 31],
          [20, 19, 19]]]])
tensor([[[[-11,  82,  30],
          [ 69,  42,  36],
          [ 21,  72,   0]]]])
tensor([[[[  0,  72,  21],
          [ 36,  42,  69],
          [ 30,  82, -11]]]])


In [80]:
print(C(C(a, k1_11), k2_11))
print(C(C(a, k2_11), k1_11))

tensor([[[[162, 205, 127],
          [261, 357, 199],
          [176, 209, 117]]]])
tensor([[[[162, 205, 127],
          [261, 357, 199],
          [176, 209, 117]]]])


In [51]:
print(C(k2_11, k1_11))
print(C(k1_11, k2_11))

tensor([[[[ 8, 11,  8],
          [ 7,  6,  7],
          [ 8, 11,  8]]]])
tensor([[[[ 8, 11,  8],
          [ 7,  6,  7],
          [ 8, 11,  8]]]])


In [53]:
C(k1_11, a)

tensor([[[[ 7, 19, 26],
          [17, 27, 31],
          [29, 19, 19]]]])

In [86]:
# associative - no
print(C(C(a, k1_11), k2_11))
print(C(a, C(k1_11, k2_11)))

# associative - no
print(C(C(k1_11, k1_11), k2_11))
print(C(k1_11, C(k1_11, k2_11)))

print(a)
print(k1_11)
print(k2_11)
print(C(k1_11, k2_11))
print(C(a, k1_11))

tensor([[[[162, 205, 127],
          [261, 357, 199],
          [176, 209, 117]]]])
tensor([[[[165, 201, 112],
          [273, 357, 187],
          [215, 213,  90]]]])
tensor([[[[34, 44, 34],
          [36, 48, 36],
          [34, 44, 34]]]])
tensor([[[[34, 44, 34],
          [36, 48, 36],
          [34, 44, 34]]]])
tensor([[[[ 2,  5,  6],
          [10,  1,  3],
          [ 8,  7,  0]]]])
tensor([[[[0, 1, 0],
          [1, 2, 1],
          [0, 1, 0]]]])
tensor([[[[3, 2, 3],
          [0, 1, 0],
          [3, 2, 3]]]])
tensor([[[[ 8, 11,  8],
          [ 7,  6,  7],
          [ 8, 11,  8]]]])
tensor([[[[19, 19, 20],
          [31, 27, 13],
          [33, 23, 10]]]])


In [21]:
import time
import cv2
import numpy as np
from scipy import signal
from scipy import ndimage

import pdb

In [41]:

def convolve_1(image, kernel):
    return cv2.filter2D(image, cv2.CV_64FC1, cv2.flip(kernel, -1), borderType=cv2.BORDER_CONSTANT)


def convolve_2(image, kernel):
    return signal.convolve2d(image, kernel, mode='full', boundary='fill', fillvalue=0)


def convolve_3(image, kernel):
    return signal.fftconvolve(image, kernel, mode='full')


def convolve_4(image, kernel):
    return signal.oaconvolve(image, kernel, mode='full')


def convolve_5(image, kernel):
    return ndimage.convolve(image, kernel, mode='constant', cval=0)

def convolve_6(image, kernel):
    if len(image.shape) < 4:
        image = torch.tensor(image).unsqueeze(0).unsqueeze(0)
    if len(kernel.shape) < 4:
        kernel = torch.tensor(kernel).unsqueeze(0).unsqueeze(0)
    return C(image, kernel)


CONVOLUTION_FUNCTIONS = {
    'cv2.filter2D': convolve_1,
    'scipy.signal.convolve2d': convolve_2,
    'scipy.signal.fftconvolve': convolve_3,
    'scipy.signal.oaconvolve': convolve_4,
    'scipy.ndimage.convolve': convolve_5,
    'torch.nn.conv2d': convolve_6
}


image = cv2.imread('/home/harishbabu/data/CUB_27/dataset/test_crop/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg',
                    cv2.IMREAD_GRAYSCALE).astype(np.float64)
kernel_1 = np.array([[0, -1, 0],
                    [-1, 5, -1],
                    [0, -1, 0]], dtype=np.float64)
kernel_2 = np.array([[ 0,  0, -1,  0,  0],
                    [ 0, -1, -2,  0,  0],
                    [-1, -2, 16, -2, -1],
                    [ 0, -1, -2, -1,  0],
                    [ 0,  0, -1,  0,  0]], dtype=np.float64)


for function_name, convolve in CONVOLUTION_FUNCTIONS.items():
    print(function_name)
    start = time.perf_counter()
    
    # these results should all be the same
    result_1 = convolve(image, kernel_1)
    result_1 = convolve(result_1, kernel_2)

    result_2 = convolve(image, kernel_2)
    result_2 = convolve(result_2, kernel_1)

    result_3 = convolve(image, convolve(kernel_2, kernel_1))

    if function_name == 'torch.nn.conv2d':
        result_1 = result_1.data.numpy()
        result_2 = result_2.data.numpy()
        result_3 = result_3.data.numpy()
    

    print('error 1:', np.mean(abs(result_1 - result_2)))
    print('error 2:', np.mean(abs(result_1 - result_3)))
    print('error 3:', np.mean(abs(result_2 - result_3)))

    print('time taken:', time.perf_counter() - start)
    print()


cv2.filter2D
error 1: 8.152428810720268
error 2: 599.2104137709825
error 3: 607.1789443672262
time taken: 0.0012714434415102005

scipy.signal.convolve2d
error 1: 0.0
error 2: 0.0
error 3: 0.0
time taken: 0.008900688029825687

scipy.signal.fftconvolve
error 1: 9.16870935488815e-13
error 2: 2.6839451408801327e-12
error 3: 2.643292003309874e-12
time taken: 0.003824138082563877

scipy.signal.oaconvolve
error 1: 1.2340172500903047e-12
error 2: 2.1855790406465643e-12
error 3: 2.26222945764845e-12
time taken: 0.0065751755610108376

scipy.ndimage.convolve
error 1: 8.152428810720268
error 2: 599.2104137709825
error 3: 607.1789443672262
time taken: 0.001843358390033245

torch.nn.conv2d


error 1: 8.414733240671442
error 2: 599.1032823692933
error 3: 607.2961972985495
time taken: 1.0557658756151795

